In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Carregar dataset
df = pd.read_csv("dataset.csv")

# Eliminar id
df = df.drop(columns=["id"])

# Separar target
X = df.drop(columns=["target_variable"])
y = df["target_variable"]

# One-hot encoding per variables categòriques
X = pd.get_dummies(X, drop_first=True)

# Train/test split estratificat
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Escalat (opcional per arbres, però mantinc per coherència)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model base Random Forest
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Distribució de paràmetres per RandomizedSearchCV
param_dist = {
    "n_estimators": [200, 500, 800],
    "max_depth": [None, 6, 10, 15],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None],
    "bootstrap": [True, False]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=25,
    scoring="f1",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

search.fit(X_train_scaled, y_train)
best_model = search.best_estimator_

print("Best CV F1:", search.best_score_)
print("Best params:", search.best_params_)

# --- Threshold tuning ---
y_val_proba = best_model.predict_proba(X_test_scaled)[:, 1]
thresholds = np.linspace(0.1, 0.9, 81)

best_f1, best_t = -1, None
for t in thresholds:
    y_val_pred = (y_val_proba >= t).astype(int)
    f1 = f1_score(y_test, y_val_pred)
    if f1 > best_f1:
        best_f1, best_t = f1, t

print("Best threshold:", best_t)
print("Test F1 at best threshold:", best_f1)

# Altres mètriques amb el llindar òptim
y_test_pred = (y_val_proba >= best_t).astype(int)
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("Recall:", recall_score(y_test, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))


Fitting 5 folds for each of 25 candidates, totalling 125 fits
